In [1]:
# Load libraries
if (!require("pacman")) install.packages("pacman")
library(pacman)
pacman::p_load("tidyverse", "psych", "data.table",  # Data wrangling and descriptive stats
               "epiDisplay", "sjstats")             # Reporting Odds Ratio and Std. Betas


# Load full dataset
df_clean <- read.csv("X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/AP_processed/df_clean.csv")
head(df_clean)

Loading required package: pacman
Warning message:
"package 'pacman' was built under R version 3.6.3"Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"

package 'epiDisplay' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\feng8\AppData\Local\Temp\RtmpOAjDVs\downloaded_packages



epiDisplay installed
Warning message:
"package 'epiDisplay' was built under R version 3.6.3"also installing the dependencies 'rlang', 'estimability', 'mvtnorm', 'minqa', 'nloptr', 'statmod', 'RcppEigen', 'bayestestR', 'dplyr', 'effectsize', 'emmeans', 'insight', 'lme4', 'parameters', 'performance', 'sjlabelled', 'sjmisc'

Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"Warning message:
"package 'dplyr' is in use and will not be installed"

package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying D:\Anaconda3\envs\R-tutorial\Lib\R\library\00LOCK\rlang\libs\x64\rlang.dll to D:\Anaconda3\envs\R-tutorial\Lib\R\library\rlang\libs\x64\rlang.dll: Permission denied"Warning message:
"restored 'rlang'"

package 'estimability' successfully unpacked and MD5 sums checked
package 'mvtnorm' successfully unpacked and MD5 sums checked
package 'minqa' successfully unpacked and MD5 sums checked
package 'nloptr' successfully unpacked and MD5 sums checked
package 'statmod' successfully unpacked and MD5 sums checked
package 'RcppEigen' successfully unpacked and MD5 sums checked
package 'bayestestR' successfully unpacked and MD5 sums checked
package 'effectsize' successfully unpacked and MD5 sums checked
package 'emmeans' successfully unpacked and MD5 sums checked
package 'insight' successfully unpacked and MD5 sums checked
package 'lme4' successfully unpacked and MD5 sums checked
package 'parameters' successfully unpacked and MD5 sums checked
package 'performance' successfully unpacked and MD5 sums checked
package 'sjlabelled' successfully unpacked and MD5 sums checked
package 'sjmisc' successfully unpacked and MD5 sums checked
package 'sjstats' successfully unpacked and MD5 sums checked

The dow


sjstats installed
Warning message:
"package 'sjstats' was built under R version 3.6.3"Warning message in pacman::p_load("tidyverse", "psych", "data.table", "epiDisplay", :
"Failed to install/load:
sjstats"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,...,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term,summer_crs,enrl_from_cohort,numgrade,honor_1,skipped_course,discipline
2607686,0,Asian,2,1,0,0,0,1,2013,...,0,1.00,2.3,4152,0,1.33,0.7,0,0,BIO
2609268,1,White,0,0,0,0,0,1,2013,...,0,0.00,4.0,4152,0,1.33,3.3,0,0,BIO
2615928,1,White,0,1,1,0,0,1,2013,...,0,1.33,4.0,4158,0,2.00,4.0,0,0,BIO
2674044,0,White,0,1,1,0,0,1,2014,...,NA,NA,NA,4152,0,0.33,3.7,0,1,BIO
2743869,0,White,0,1,1,0,0,1,2014,...,0,1.00,3.7,4162,0,1.33,3.0,0,0,BIO
2744813,0,White,0,1,0,0,0,1,2014,...,0,0.33,2.3,4162,0,1.33,NA,0,0,BIO


In [2]:
# Subsets data for analysis 
# Took 2nd course in sequence
df_bio2 <- df_clean %>%
  subset(discipline == "BIO") 
# Took AP 
df_bio_aptakers <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(aptaker == 1)
# Skip eligible
df_bio_skeligible <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(eligible_to_skip == 1)

# Chem
# Took 2nd course in sequence
df_chem2 <- df_clean %>%
  subset(discipline == "CHEM") 
# Took AP 
df_chem_aptakers <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(aptaker == 1)
# Skip eligible
df_chem_skeligible <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(eligible_to_skip == 1)

# Phys
# Took 2nd course in sequence
df_phys2 <- df_clean %>%
  subset(discipline == "PHYS") 
# Took AP 
df_phys_aptakers <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(aptaker == 1)
# Skip eligible
df_phys_skeligible <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(eligible_to_skip == 1)